In [23]:
import sys
import numpy as np
from random import randint
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import operator
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 

### Loading Data from training.csv and creating z1 , z2 which are X_training and X_test respectively.

In [2]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [24]:
file = r'/Users/shikhgar/Downloads/trainingandtestdata/training.csv'
df = pd.read_csv(file,encoding = "ISO-8859-1",names = ["sentiment", "a", "b", "c" ,"d","tweet"])
data = df.iloc[:,[0,5]]
temp = list(data["tweet"])
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer() 
corpus2 = temp[0:20000] + temp[-20000:]
corpus = []
for tweet in corpus2:
    corpus.append(lemmatizer.lemmatize((pre_process(tweet))))
target = np.zeros((40000,1))
target[20000:,0] = 1

In [25]:
print(temp[0])
print(temp2[0])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
 switchfoot http twitpic com y zl awww that s a bummer you shoulda got david carr of third day to do it d


In [26]:
stop_words = ['in', 'of', 'at', 'a', 'the']
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words='english')
X = vectorizer.fit_transform(corpus)

In [27]:
X_tr, X_te, Y_train, Y_test = train_test_split(
    X, target, train_size = 0.75
)

In [28]:
z1 = (X_tr)
z2 = (X_te)
print(z1.shape, z2.shape)
n = z1.shape[1]

(30000, 432638) (10000, 432638)


In [29]:
def extract_subset(z,cols):
    return z[:,cols]

In [30]:
def svm_calc_fitness(x):
    SzW = 0.01
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = svm.LinearSVC(C=0.1)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test, y_pred)
    fscore = sklearn.metrics.f1_score(Y_test, y_pred)
    recall = sklearn.metrics.recall_score(Y_test, y_pred)
    return ((1 - SzW) * (1 - acc)) + (SzW * (sum(x) / (len(x) - 1))),pre,fscore,recall

In [31]:
def rf_calc_fitness(x):
    SzW = 0.01
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = RandomForestClassifier(n_jobs=2,random_state=0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test, y_pred)
    fscore = sklearn.metrics.f1_score(Y_test, y_pred)
    recall = sklearn.metrics.recall_score(Y_test, y_pred)
    return ((1 - SzW) * (1 - acc)) + (SzW * (sum(x) / (len(x) - 1))),pre,fscore,recall

In [32]:
def knn_calc_fitness(x):
    SzW = 0.01
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = KNN()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return ((1 - SzW) * (1 - acc)) + (SzW * (sum(x) / (len(x) - 1))),pre,fscore,recall

In [33]:
def nb_calc_fitness(x):
    SzW = 0.01
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = GNB()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return ((1 - SzW) * (1 - acc)) + (SzW * (sum(x) / (len(x) - 1))),pre,fscore,recall

In [34]:
def mlp_calc_fitness(x):
    SzW = 0.01
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    scaler = StandardScaler()
    scaler.fit(z1_prime)
    z1_prime = scaler.transform(z1_prime)
    z2_prime = scaler.transform(z2_prime)
    clf = MLPClassifier()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return ((1 - SzW) * (1 - acc)) + (SzW * (sum(x) / (len(x) - 1))),pre,fscore,recall

In [35]:
#m = population n = features
def sigmoid (x,l):
    return (1/(1+math.exp(-l[0]*(x-l[1]))))

In [36]:
def CrossOver(x1,x2,x3):
    r = random.uniform(0,1)
    if r < 0.333:
        return x1
    elif r<0.6666:
        return x2
    else :
        return x3

In [37]:
def initialise(m,n):
    pos = defaultdict(list)
    for i in range(0,m):
        for j in range(0,n):
            pos[i].append(randint(0,1))
    return pos

In [38]:
def BGWO(m,T,n):

    infinity = float("inf")
    AlphaScore = infinity
    BetaScore = infinity
    DeltaScore = infinity

    AlphaPos = [0]*n
    BetaPos = [0]*n
    DeltaPos = [0]*n

    Positions = initialise(m,n);

    convergence_curve = [0] * T

    for t in range(0,T):
        print(t)
        for i in range(0,m):
            fitness,pre,fscore,recall = svm_calc_fitness(Positions[i])
            if fitness < AlphaScore:
                AlphaScore = fitness
                AlphaPos = Positions[i]

            if fitness > AlphaScore and fitness < BetaScore:
                BetaScore = fitness
                BetaPos = Positions[i]

            if fitness > AlphaScore and fitness > BetaScore and fitness < DeltaScore:
                DeltaScore = fitness
                DeltaPos = Positions[i]

            #with open("GWO_svm_kpi.txt", "a") as text_file:
                #text_file.write("{} {} {} {}".format(fitness,pre,fscore,recall) + "\n")

            with open("GWO_svm.txt", "a") as text_file:
                text_file.write("{} {} {} {} {} {}".format(t, fitness,sum (1 for i in Positions[i] if i > 0.5), 1-AlphaScore,1-BetaScore,1-DeltaScore)+"\n")

            #print("t={} fitness={} #features={} AlphaScore={} ".format(t, fitness,sum (1 for i in Positions[i] if i > 0.5), 1-AlphaScore))

        a = 2 - t*((2)/T)
        for i in range(0,m):
            for j in range(0,n):

                r1 = random.uniform(0,1)
                r2 = random.uniform(0,1)
                sigma = random.uniform(0,1)

                A1 = 2 * a * r1 - a
                C1 = 2 * r2

                D_alpha = abs(C1*AlphaPos[j] - Positions[i][j])
                v1 = sigmoid(-A1*D_alpha,[10,0.5])

                if v1 < sigma:
                    v1=0
                else :
                    v1 = 1
                X1 = (AlphaPos[j]+v1)>=1

                r1 = random.uniform(0, 1)
                r2 = random.uniform(0, 1)
                sigma = random.uniform(0, 1)

                A2 = 2 * a * r1 - a
                C2 = 2 * r2

                D_beta = abs(C2 * BetaPos[j] - Positions[i][j])
                v1 = sigmoid(-A2 * D_beta, [10, 0.5])

                if v1 < sigma:
                    v1 = 0
                else:
                    v1 = 1
                X2 = (BetaPos[j] + v1) >= 1

                r1 = random.uniform(0, 1)
                r2 = random.uniform(0, 1)
                sigma = random.uniform(0, 1)

                A3 = 2 * a * r1 - a
                C3 = 2 * r2

                D_delta = abs(C3 * DeltaPos[j] - Positions[i][j])
                v1 = sigmoid(-A3 * D_delta, [10, 0.5])

                if v1 < sigma:
                    v1 = 0
                else:
                    v1 = 1
                X3 = (DeltaPos[j] + v1) >= 1

                Positions[i][j] = CrossOver(X1,X2,X3)

        convergence_curve[t]=AlphaScore

    return convergence_curve,AlphaScore

In [ ]:
ans,score = BGWO(70,50,n) #m,T,n   T =  Iterations(100) , n= features(dont change) m = agents (50-70)
print(score)

0


/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
